In [ ]:
!pip install "numpy<1.24.0"
!pip install --upgrade pip
!pip install hdbscan
!pip install umap-learn
!pip install spacy-langdetect
!pip install language-detector
!pip install symspellpy
!pip install sentence-transformers
!pip install stop-words
!pip install --upgrade pip
!pip install hdbscan

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input/mallet'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

import json
from tqdm import tqdm
from nltk.corpus import wordnet
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import datetime
import time
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import nltk

In [77]:
fin='triple_long_sf_Vax'
filein="/content/"+str(fin)+".csv"
meta = pd.read_csv(filein, header=0, encoding='ISO-8859-1', engine='python')

print(meta.shape)
print(len(meta))

(13806, 22)
13806


In [78]:
meta=meta.drop_duplicates(subset=['triple_string'], keep='first')

In [79]:
print(len(meta))

13806


In [80]:
print(meta.head())

   postInd  sentenceInd  tripleInd          subject  \
0        0            0          0              fda   
1        1            0          0            trump   
2        1            1          0  first americans   
3        1            1          5               us   
4        2            0          0               it   

                                            relation  \
0  authorizes pfizerbiontech coronavirus vaccine for   
1                                          announces   
2                                               will   
3                                         authorised   
4                                            receive   

                                          object  \
0                                  emergency use   
1                                vaccine rollout   
2  vaccinated against coronavirus in next 24 hrs   
3      pfizer/biontech vaccine for emergency use   
4                                approval in usa   

                   

In [81]:
#meta["triple_string"] = meta['subject'] + " " + meta['relation'] + " " + meta['object']

In [82]:
print(meta["triple_string"].head())

0    fda authorizes pfizerbiontech coronavirus vacc...
1                      trump announces vaccine rollout
2    first americans will vaccinated against corona...
3    us authorised pfizer/biontech vaccine for emer...
4                           it receive approval in usa
Name: triple_string, dtype: object


In [83]:
from collections import Counter
from sklearn.metrics import silhouette_score
import umap.umap_ as umap
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
import os

def get_topic_words(token_lists, labels, k=None):
  #get top words within each topic from clustering results
  if k is None:
    k = len(np.unique(labels))
  topics = ['' for _ in range(k)]
  for i, c in enumerate(token_lists):
    topics[labels[i]] += (' ' + ' '.join(c))
  
  word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
  # get sorted word counts
  word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
  # get topics
  topics = list(map(lambda x: list(map(lambda x: x[0], x[:15])), word_counts))

  return topics

def get_coherence(model, token_lists, measure='c_v'):
  if model.method == 'LDA':
    cm = CoherenceModel(model=model.ldamodel, texts=token_lists, 
                  corpus=model.corpus, dictionary=model.dictionary,
                      coherence=measure)
  else:
    topics = get_topic_words(token_lists, model.cluster_model.labels_)
    cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, 
                        dictionary=model.dictionary, coherence=measure)
  
  return cm.get_coherence()

def get_silhouette(model):
  if model.method == 'LDA':
      return
  lbs = model.cluster_model.labels_
  vec = model.vec[model.method]
  
  return silhouette_score(vec, lbs)


In [84]:
import nltk, re, string
nltk.download('stopwords')

from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.tokenize import word_tokenize
from language_detector import detect_language

import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")

if sym_spell.word_count:
  pass
else:
  sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

def preprocess_sent(rw):
  s = rw

  return s

def preprocess_word(s):
  stop_words = set(stopwords.words('english'))

  if not s :
    return None
  
  w_list = word_tokenize(s)

  res_list=[]

  for t in w_list:
    if t not in stop_words:
      res_list.append(re.sub(r"\d+", "", t))
    else:
      res_list.append("")

  res_list=[(re.sub(" +", " ", (t))) for t in res_list]
  
  res_list2=[]
  for t in res_list:
    t = re.sub(r"[{}]".format(string.punctuation), "", t)
    
    res_list2.append(t)

  return res_list2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

class Autoencoder:
  def __init__(self, latent_dim=32, activation='relu', epochs=300, batch_size=128):
    self.latent_dim = latent_dim
    self.activation = activation
    self.epochs = epochs
    self.batch_size = batch_size
    self.autoencoder = None
    self.encoder = None
    self.decoder = None
    self.his = None

  def _compile(self, input_dim):
    input_vec = Input(shape=(input_dim,))
    encoded = Dense(self.latent_dim, activation=self.activation)(input_vec)
    decoded = Dense(input_dim, activation=self.activation)(encoded)
    self.autoencoder = Model(input_vec, decoded)
    self.encoder = Model(input_vec, encoded)
    encoded_input = Input(shape=(self.latent_dim,))
    decoder_layer = self.autoencoder.layers[-1]
    self.decoder = Model(encoded_input, self.autoencoder.layers[-1](encoded_input))
    self.autoencoder.compile(optimizer='adam', loss=keras.losses.mean_squared_error)

  def fit(self, X):
    if not self.autoencoder:
      self._compile(X.shape[1])
    
    X_train, X_test = train_test_split(X)
    self.his = self.autoencoder.fit(X_train, X_train, epochs=300, batch_size=128, shuffle=True, validation_data=(X_test, X_test), verbose=0)

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
from datetime import datetime

def preprocess(docs, samp_size=None):
  if not samp_size:
    samp_size = len(docs)

  print('Preprocessing raw texts ...')
  
  n_docs = len(docs)
  print('docs len: ', n_docs)

  sentences = []  # sentence level preprocessed
  token_lists = []  # word level preprocessed

  for sent in docs:
    sentence = preprocess_sent(sent)
    token_list = preprocess_word(sentence)
    
    if token_list:  
      sentences.append(sentence)
      token_lists.append(token_list)

  print('Preprocessing raw texts. Done!')
  
  return sentences, token_lists

# define model object
class Topic_Model:
  def __init__(self, k=10, method='TFIDF'):
    if method not in {'TFIDF', 'LDA', 'BERT', 'LDA_BERT'}:
      raise Exception('Invalid method!')
    
    self.k = k
    self.dictionary = None
    self.corpus = None
    #self.stopwords = None
    self.cluster_model = None
    self.ldamodel = None
    self.vec = {}
    self.gamma = 15  # parameter for reletive importance of lda
    self.method = method
    self.AE = None
    self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

  def vectorize(self, sentences, token_lists, method=None):
    # Default method
    if method is None:
      method = self.method

    # turn tokenized documents into a id <-> term dictionary
    self.dictionary = corpora.Dictionary(token_lists)
    # convert tokenized documents into a document-term matrix
    self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

    if method == 'TFIDF':
      print('Getting vector representations for TF-IDF ...')
      tfidf = TfidfVectorizer()
      vec = tfidf.fit_transform(sentences)
      print('Getting vector representations for TF-IDF. Done!')
      return vec

    elif method == 'LDA':
      print('Getting vector representations for LDA ...')
      if not self.ldamodel:
        self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics = self.k, id2word = self.dictionary,
                                passes=300, alpha=0.1, random_state=100)
      def get_vec_lda(model, corpus, k):
        n_doc = len(corpus)
        vec_lda = np.zeros((n_doc, k))
        for i in range(n_doc):
            # get the distribution for the i-th document in corpus
          for topic, prob in model.get_document_topics(corpus[i]):
            vec_lda[i, topic] = prob

        return vec_lda

      vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
      print('Getting vector representations for LDA. Done!')

      return vec

    elif method == 'BERT':
      print('Getting vector representations for BERT ...')
      from sentence_transformers import SentenceTransformer
      model = SentenceTransformer('bert-base-nli-max-tokens')
      vec = np.array(model.encode(sentences, show_progress_bar=True))
      print('Getting vector representations for BERT. Done!')

      return vec

    elif method == 'LDA_BERT':
      global vec_lda 
      vec_lda = self.vectorize(sentences, token_lists, method='LDA')
      print("LDA Vector and shape:")
      print(vec_lda)
      print(vec_lda.shape)
      #d['vec_lda']=vec_lda

      global vec_bert
      vec_bert = self.vectorize(sentences, token_lists, method='BERT')
      print("Bert vector and shape:")
      print(vec_bert)
      print(vec_bert.shape)
      #d['vec_bert']=vec_bert

      global vec_ldabert
      vec_ldabert = np.c_[vec_lda * self.gamma, vec_bert]
      print("Concatenated Vector and shape:")
      print(vec_ldabert)
      print(vec_ldabert.shape)
      #d['vec_ldabert']=vec_ldabert
      self.vec['LDA_BERT_FULL'] = vec_ldabert
      
      if not self.AE:
        self.AE = Autoencoder()
        print('Fitting Autoencoder ...')
        self.AE.fit(vec_ldabert)
        print('Fitting Autoencoder Done!')
      
      global vec_lowered
      vec_lowered = self.AE.encoder.predict(vec_ldabert)
      print("Encoder vector and shape")
      print(vec_lowered)
      print(vec_lowered.shape)
      #meta['low_vec']=vec
      return vec_lowered

  def fit(self, sentences, token_lists, method=None, m_clustering=None):
    # Default method
    if method is None:
      method = self.method
    # Default clustering method
    if m_clustering is None:
      m_clustering = KMeans

    # turn tokenized documents into a id <-> term dictionary
    if not self.dictionary:
      self.dictionary = corpora.Dictionary(token_lists)
      # convert tokenized documents into a document-term matrix
      self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

    if method == 'LDA':
      if not self.ldamodel:
        print('Fitting LDA ...')
        self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics = self.k, id2word = self.dictionary,
                                passes=300, alpha=0.1, random_state=100)
        print('Fitting LDA Done!')
    else:
      print('Clustering embeddings ...')
      self.cluster_model = m_clustering(self.k)
      self.vec[method] = self.vectorize(sentences, token_lists, method)
      self.cluster_model.fit(self.vec[method])
      print('Clustering embeddings. Done!')

  def predict(self, sentences, token_lists, out_of_sample=None):
    # Default as False
    out_of_sample = out_of_sample is not None

    if out_of_sample:
      print("out_of_sample")
      corpus = [self.dictionary.doc2bow(text) for text in token_lists]
      if self.method != 'LDA':
        vec = self.vectorize(sentences, token_lists)
        print(vec)
    else:
      print("Not out_of_sample")
      corpus = self.corpus
      vec = self.vec.get(self.method, None)

    if self.method == "LDA":
      print("LDA")
      lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x), key=lambda x: x[1], reverse=True)[0][0], corpus)))
    else:
      print("Not LDA")
      lbs = self.cluster_model.predict(vec)
    
    print(lbs)

    return lbs

In [87]:
###########################

In [88]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=Warning)

import argparse

def main():
  method = "LDA_BERT"
  
  ntopic = 3
  
  d=meta
  d=d.reset_index()
  print("data length:\t", d, '\n\n')

  data = d
  data = data.fillna('')  # only the comments has NaN's  

  rws = data.triple_string

  sentences, token_lists = preprocess( rws, samp_size = len(rws) )

  tm = Topic_Model(k = ntopic, method = method)
  tm.fit(sentences, token_lists)

  print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
  print('Silhouette Score:', get_silhouette(tm))
  
  return (tm, ntopic, token_lists)

In [89]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [90]:
(tm, k, token_lists) = main()

data length:	        index  postInd  sentenceInd  tripleInd  \
0          0        0            0          0   
1          1        1            0          0   
2          2        1            1          0   
3          3        1            1          5   
4          4        2            0          0   
...      ...      ...          ...        ...   
13801  13801    18018            0          0   
13802  13802    18021            0          1   
13803  13803    18022            0          0   
13804  13804    18023            1          6   
13805  13805    18023            1          8   

                                     subject  \
0                                        fda   
1                                      trump   
2                            first americans   
3                                         us   
4                                         it   
...                                      ...   
13801                         healthy people   
13802  covid-

Batches:   0%|          | 0/432 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
Bert vector and shape:
[[ 0.5867184   0.63556975  0.9927522  ...  0.2678192   0.6252658
   0.39211506]
 [ 1.0291862   0.5893801   1.0717392  ...  0.23634212  1.0159149
   0.30808556]
 [ 0.8049095   0.31052312  1.4356116  ... -0.3794963   1.1795747
   0.62937826]
 ...
 [ 0.7641528   0.35097498  0.6440487  ...  1.3221947   1.6104041
  -0.26220846]
 [ 0.9424208   0.06836711  0.91814464 ...  0.09663146 -0.22041179
  -0.04379624]
 [ 0.9779019   0.86818314  0.7901161  ... -0.00953531  0.19540045
   0.5999408 ]]
(13806, 768)
Concatenated Vector and shape:
[[ 1.80740543e-01  1.80760422e-01  1.46384990e+01 ...  2.67819196e-01
   6.25265777e-01  3.92115057e-01]
 [ 3.48854670e-01  1.43022233e+01  3.48922592e-01 ...  2.36342117e-01
   1.01591492e+00  3.08085561e-01]
 [ 9.46667522e+00  5.38764596e+00  0.00000000e+00 ... -3.79496306e-01
   1.17957473e+00  6.29378259e-01]
 ...
 [ 1.43022627e+01  3.48838074e-01  3.48899094e-01 ...  1.32219470e+00
   1.610

In [91]:
################################################################################

In [92]:
from nltk.cluster import KMeansClusterer
import nltk

d=meta
d=d.reset_index()

In [93]:
d2 = pd.DataFrame()
d2 = meta

#d2['topic'] = resPrediction
d2['vec_lowered'] = vec_lowered.tolist()

In [94]:
#d2.to_csv(str(fin)+"_embs.csv", index = None, header=True)

In [95]:
! python -m pip install --upgrade pip --quiet
! pip install spacy --quiet
! pip3 install spacy-transformers --quiet
! python -m spacy download en_trf_bertbaseuncased_lg --quiet

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-03 04:42:33.344304: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-03 04:42:33.344427: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-03 04:42:33.344451: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [96]:
df2=d2

In [97]:
from nltk.cluster import KMeansClusterer
import nltk, re
from scipy.spatial import distance_matrix


centroid=[]
c=0
newls=[]

for (l1) in list(df2['vec_lowered']):
  ls=[]

  l1=str(l1)
  tt=l1
  tt=re.sub("\[", "", tt)
  tt=re.sub("\]", "", tt)

  ll=tt.split(", ")

  for s in ll:
    ls.append(float(s))
  newls.append(ls)

  c+=1

X = np.array(newls)
Y = np.mean(X, axis=0)

print(Y, '\n')

centroid.append(Y)
#########################################################################################################################################################

[0.00000000e+00 1.76318564e+00 1.24288432e+00 1.18603162e+00
 1.65360431e+00 2.18852948e+00 1.66722911e+00 1.60016312e+00
 1.20951030e+00 2.84424170e+00 0.00000000e+00 1.71366072e+00
 1.59233603e+00 0.00000000e+00 0.00000000e+00 1.37217675e+00
 1.68521405e+00 2.50745546e+00 7.29369801e-06 1.25870874e+00
 0.00000000e+00 3.10916085e-06 2.23218543e+00 1.43970942e+00
 1.26788560e+00 1.72744485e+00 1.52179057e+00 1.76918992e+00
 1.32504695e+00 1.38777067e+00 0.00000000e+00 2.75731972e+00] 



In [98]:
Z=[]

for c in df2['vec_lowered']:
  Z.append(centroid[0])

print(len(Z))

13806


In [99]:
df2['centroid']=Z

In [100]:
Slist=[]

for ll in df2['centroid']:
  slist=[]
  for i in ll:
    slist.append(float(i))

  Slist.append(slist)
  
df2['centroid']=Slist

In [101]:
df2=df2.sort_values(by = ['postInd', 'sentenceInd', 'tripleInd'], ascending = [True, True, True])

In [102]:
df1=df2

In [103]:
from scipy.spatial import distance_matrix
import re

newls=[]
for (l1, ll2) in zip(df1['vec_lowered'].tolist(), df1['centroid'].tolist()):
    ls=[]
    l1=str(l1)
    tt=l1
    tt=re.sub("\[", "", tt)
    tt=re.sub("\]", "", tt)
    ll=tt.split(", ")
    for s in ll:
        ls.append(float(s))
    
    X= np.array(ls)
    
    ls2=[]
    l12=str(ll2)
    tt2=l12
    tt2=re.sub("\[", "", tt2)
    tt2=re.sub("\]", "", tt2)
    ll2=tt2.split(", ")
    for s2 in ll2:
        ls2.append(float(s2))
    
    Y = np.array(ls2)
    print(distance_matrix( [X], [Y] ))
    
    a=str(distance_matrix( [X], [Y] ))
    a=re.sub('\[', '', a)
    a=re.sub('\]', '', a)
    a=float(a)
    newls.append(a)
    print(a, '\n')
    
df1['distance_from_centroid']=newls

串流輸出內容已截斷至最後 5000 行。
5.14275713 

[[5.39582063]]
5.39582063 

[[3.36368401]]
3.36368401 

[[3.91615833]]
3.91615833 

[[3.4204862]]
3.4204862 

[[4.57529128]]
4.57529128 

[[4.57342422]]
4.57342422 

[[4.24325923]]
4.24325923 

[[4.79372844]]
4.79372844 

[[4.17328677]]
4.17328677 

[[3.82267875]]
3.82267875 

[[4.59738941]]
4.59738941 

[[5.38798005]]
5.38798005 

[[5.85253264]]
5.85253264 

[[5.84446373]]
5.84446373 

[[4.83160745]]
4.83160745 

[[4.63215342]]
4.63215342 

[[4.65150496]]
4.65150496 

[[4.27850064]]
4.27850064 

[[4.05875426]]
4.05875426 

[[3.98939378]]
3.98939378 

[[5.50332054]]
5.50332054 

[[3.84296118]]
3.84296118 

[[6.01020242]]
6.01020242 

[[4.81727819]]
4.81727819 

[[4.90133869]]
4.90133869 

[[5.25318756]]
5.25318756 

[[4.52499932]]
4.52499932 

[[5.48765898]]
5.48765898 

[[4.85792408]]
4.85792408 

[[3.95043095]]
3.95043095 

[[4.15770895]]
4.15770895 

[[5.63950855]]
5.63950855 

[[4.45118845]]
4.45118845 

[[4.03374516]]
4.03374516 

[[3.4042794]]
3.

In [104]:
df1=df1.sort_values(by = ['postInd', 'sentenceInd', 'tripleInd'], ascending = [True, True, True])

In [105]:
df1.to_csv(str(fin)+'_embs_cen_d.csv', index = None, header=True)

In [106]:
######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################
######################################################################################################################

In [107]:
df1 = df1.sort_values(by = ['postInd', 'sentenceInd', 'tripleInd'], ascending = [True, True, True])

In [108]:
print(len(df1))

13806


In [109]:
from pandas import DataFrame
dout=DataFrame()

dt=df1.sort_values(["distance_from_centroid"], ascending=[True])#.head(21)
print(len(dt))

dout=pd.concat([dout, dt], axis=0)
print(len(dout))
print("**************************************")

13806
13806
**************************************


In [110]:
print(dout)

       postInd  sentenceInd  tripleInd                        subject  \
7044      8911            0          3                   central govt   
9359     12045            0          1                      indonesia   
8211     10525            0          2                nstworld israel   
13032    16878            0         11  earlier gamaleya center study   
10938    14081            0          0                         cdcgov   
...        ...          ...        ...                            ...   
13355    17382            1          0                             we   
2182      2483            0          2                             it   
10012    12887            2          0                    vaccination   
1703      1929            0          0                              i   
298        383            1          0                       everyone   

                                      relation  \
7044                             recently gave   
9359                   

In [111]:
dout=dout.sort_values(by = ['postInd', 'sentenceInd', 'tripleInd'], ascending = [True, True, True])

print(dout)

#dout.to_csv(str(fin)+'_topSentences' + '.csv', header=True, index=None)

       postInd  sentenceInd  tripleInd                                subject  \
0            0            0          0                                    fda   
1            1            0          0                                  trump   
2            1            1          0                        first americans   
3            1            1          5                                     us   
4            2            0          0                                     it   
...        ...          ...        ...                                    ...   
13801    18018            0          0                         healthy people   
13802    18021            0          1  covid-19 vaccine covaxin true example   
13803    18022            0          0                       malaysia reserve   
13804    18023            1          6                                    its   
13805    18023            1          8                                    its   

                           